In [1]:
import keras
from keras.datasets import mnist
# the data, shuffled and split between train and test sets
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print(Y_train[:10])

Using TensorFlow backend.


60000 train samples
10000 test samples
[5 0 4 1 9 2 1 3 1 4]


In [7]:
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout
# global optimization to find coefficients for weighted ensemble on blobs problem
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from scipy.optimize import differential_evolution

# fit model on dataset
def fit_model(trainX, trainy, testX, testY):
    
    batch_size = 128
    # define model
    model = Sequential()
    model.add(Dense(512, input_dim=784, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    trainy_enc = to_categorical(trainy)
    test_enc = to_categorical(testY)
    model.fit(trainX, trainy_enc, batch_size=batch_size,
              epochs=10,
              verbose=1,
              validation_data=(testX, test_enc))
    return model


# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = array(yhats)
    # weighted sum across ensemble members
    summed = tensordot(yhats, weights, axes=((0),(0)))
    # argmax across classes
    result = argmax(summed, axis=1)
    return result

# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testy):
    # make prediction
    yhat = ensemble_predictions(members, weights, testX)
    # calculate accuracy
    return accuracy_score(testy, yhat)

# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result

# loss function for optimization process, designed to be minimized
def loss_function(weights, members, testX, testy):
    # normalize weights
    normalized = normalize(weights)
    # calculate error rate
    return 1.0 - evaluate_ensemble(members, normalized, testX, testy)

In [8]:
n_members = 5 #no of stacked models
members = [
    fit_model(X_train, Y_train, X_test, Y_test) for _ in range(n_members)
]

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 206us/step - loss: 0.0448 - accuracy: 0.9849 - val_loss: 0.0182 - val_accuracy: 0.9936
Epoch 2/10
60000/60000 [==============================] - 11s 183us/step - loss: 0.0205 - accuracy: 0.9932 - val_loss: 0.0160 - val_accuracy: 0.9945
Epoch 3/10
60000/60000 [==============================] - 11s 183us/step - loss: 0.0154 - accuracy: 0.9948 - val_loss: 0.0139 - val_accuracy: 0.9953
Epoch 4/10
60000/60000 [==============================] - 11s 184us/step - loss: 0.0129 - accuracy: 0.9958 - val_loss: 0.0146 - val_accuracy: 0.9952
Epoch 5/10
60000/60000 [==============================] - 11s 185us/step - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.0146 - val_accuracy: 0.9952
Epoch 6/10
60000/60000 [==============================] - 12s 199us/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.0114 - val_accuracy: 0.9963
Epoch 7/10
60000/60000 [==============================] 

In [10]:
# evaluate each single model on the test set
testy_enc = to_categorical(Y_test)
for i in range(n_members):
    _, test_acc = members[i].evaluate(X_test, testy_enc, verbose=0)
    print('Model %d: %.3f' % (i+1, test_acc))

# evaluate averaging ensemble (equal weights)
weights = [1.0/n_members for _ in range(n_members)]
score = evaluate_ensemble(members, weights, X_test, Y_test)
print('Equal Weights Score: %.3f' % score)

# define bounds on each weight
bound_w = [(0.0, 1.0)  for _ in range(n_members)]

# arguments to the loss function
search_arg = (members, X_test, Y_test)

# global optimization of ensemble weights
"""
The differential_evolution() SciPy function requires that function is specified to evaluate a set of weights 
and return a score to be minimized.  We can minimize the classification error (1 – accuracy).
"""
result = differential_evolution(loss_function, bound_w, search_arg, maxiter=10, tol=1e-7)

# get the chosen weights
weights = normalize(result['x'])
print('Optimized Weights: %s' % weights)
# evaluate chosen weights(weights obtained from optimization)
score = evaluate_ensemble(members, weights, X_test, Y_test)
print('Optimized Weights Score: %.3f' % score)

Model 1: 0.997
Model 2: 0.996
Model 3: 0.996
Model 4: 0.996
Model 5: 0.997
Equal Weights Score: 0.987
Optimized Weights: [0.26677845 0.0296783  0.3296601  0.13227702 0.24160613]
Optimized Weights Score: 0.988


In [32]:
# weighted ensemble probability and uncertainity prediction
import math
import numpy as np
def ensemble_predictions_prob(members, weights, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = array(yhats)

    # weighted sum across ensemble members
    summed = tensordot(yhats, weights, axes=((0),(0)))
    
    #weighted var of ensemble members
    variance = tensordot((yhats-summed)**2, weights, axes=((0),(0)))
    return (summed, variance)

In [34]:
ensemble_prob, var = ensemble_predictions_prob(members, weights, X_test)

for it, prob, vari in zip(Y_test[:10], ensemble_prob[:10], var[:10]):
    print('actual: '+str(it)+', predicted: '+str(np.argmax(prob))+', confidence: '+str(prob[np.argmax(prob)])+', var: '+str(vari[np.argmax(prob)]))

actual: 7, predicted: 7, confidence: 0.9999995306633523, var: 1.9661382135270723e-13
actual: 2, predicted: 2, confidence: 0.9999999999999999, var: 1.2325951644078308e-32
actual: 1, predicted: 1, confidence: 0.9999991824032886, var: 1.2048720576961065e-12
actual: 0, predicted: 0, confidence: 0.9999059356697367, var: 3.400109780481828e-08
actual: 4, predicted: 4, confidence: 0.9818585128565763, var: 0.0006679036189163716
actual: 1, predicted: 1, confidence: 0.9999999681975303, var: 2.779752723714715e-15
actual: 4, predicted: 4, confidence: 0.9976635675615587, var: 9.581343838659796e-06
actual: 9, predicted: 9, confidence: 0.9992633131759838, var: 7.20725545011689e-07
actual: 5, predicted: 5, confidence: 0.8096238211252196, var: 0.09624524929470496
actual: 9, predicted: 9, confidence: 0.999982540330106, var: 8.214497314532808e-10
